In [4]:
import plotly.graph_objects as go
import networkx as nx
import plotly.offline as pyo
import random
import copy
import igraph as ig
import ipywidgets as widgets
from IPython.display import display, clear_output
#inicializacion
n_usuarios, monto_inicial = 300, 15
n_iteraciones, n_referidos, monto_referir = 3, 4, 2.5
l_enlaces = []
graphs = []
#widgets
n_usuarios_input = widgets.IntSlider(value=n_usuarios, min=1, max=800, description='n_usuarios:')
monto_inicial_input = widgets.IntSlider(value=monto_inicial, min=1, max=100, description='monto_inicial:')
n_iteraciones_input = widgets.IntSlider(value=n_iteraciones, min=1, max=10, description='n_iteraciones:')
n_referidos_input = widgets.IntSlider(value=n_referidos, min=1, max=10, description='n_referidos:')
monto_referir_input = widgets.FloatSlider(value=monto_referir, min=0.1, max=10, step=0.1, description='monto_referir:')

run_button = widgets.Button(description='Run Simulation')
#output para ver los resultados
output = widgets.Output()

def refiere(de_nodo, n_monto, graph, enlaces):
    new_node_id = max(graph) + 1
    graph[new_node_id] = {'profundidad': graph[de_nodo]['profundidad'] + 1,'referidos':0, 'monto':monto_referir}
    graph[de_nodo]['referidos'] += 1
    enlaces.append((de_nodo,new_node_id))
    return graph, enlaces
def actualiza(graph, enlaces):
    c_graph=graph.copy()
    for k in c_graph:
        r=random.random()
        if graph[k]['profundidad']!=0 and graph[k]['referidos']<4 and r>0.5:
            n_referir=max(1,int((n_referidos +1)*random.random())-graph[k]['referidos'])
            for _ in range(n_referir):
                graph, enlaces=refiere(k, monto_referir, graph, enlaces)
    return graph, enlaces

def run_simulation(btn):
    global n_usuarios, monto_inicial, n_iteraciones, n_referidos, monto_referir
    global graphs, enlaces
    n_usuarios = n_usuarios_input.value
    monto_inicial = monto_inicial_input.value
    n_iteraciones = n_iteraciones_input.value
    n_referidos = n_referidos_input.value
    monto_referir = monto_referir_input.value
    with output:
        output.clear_output()
        graph = {1: {'profundidad': 0, 'referidos':0, 'monto':0}}
        enlaces=[]
        for _ in range(n_usuarios):
            n_id = max(graph) + 1
            graph[n_id] = {'profundidad': 1,'referidos':0, 'monto':monto_inicial}
            enlaces.append((1,n_id))
        c_graph = copy.deepcopy(graph)
        c_enlace = copy.deepcopy(enlaces)
        graphs.append(c_graph)
        l_enlaces.append(c_enlace)
        for step in range(n_iteraciones):
            graph, enlaces=actualiza(graph, enlaces)
            c_graph = copy.deepcopy(graph)
            c_enlace = copy.deepcopy(enlaces)
            graphs.append(c_graph)
            l_enlaces.append(c_enlace)
        k_slider.max = len(l_enlaces)-1
    update_graph(0)

run_button.on_click(run_simulation)

input_widgets = widgets.VBox([n_usuarios_input, monto_inicial_input, n_iteraciones_input,
                              n_referidos_input, monto_referir_input, run_button])
app_layout = widgets.HBox([input_widgets, output])
display(app_layout)

def calculate_aggregated_sum(graph):
    return sum(graph[k]['monto'] for k in graph)

def update_graph(k_value):
    global l_enlaces
    if 0 <= k_value < len(l_enlaces):
        G = nx.Graph()
        G.add_edges_from(l_enlaces[k_value])
        spring_3D = nx.spring_layout(G, dim=3, seed=1515, scale=10)

        x_nodes = [spring_3D[i][0] for i in G.nodes()]
        y_nodes = [spring_3D[i][1] for i in G.nodes()]
        z_nodes = [spring_3D[i][2] for i in G.nodes()]

        edge_list = G.edges()
        x_edges, y_edges, z_edges = [], [], []

        for edge in edge_list:
            x_coords = [spring_3D[edge[0]][0], spring_3D[edge[1]][0], None]
            x_edges += x_coords
            y_coords = [spring_3D[edge[0]][1], spring_3D[edge[1]][1], None]
            y_edges += y_coords
            z_coords = [spring_3D[edge[0]][2], spring_3D[edge[1]][2], None]
            z_edges += z_coords

        trace_nodes = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers', marker=dict(
        symbol='circle',  # Use 'circle' symbol
        size=[5 for _ in list(G.nodes())],
        color=1,
        colorscale='Viridis',
        line=dict(
            color='black',  # Border color
            width=2          # Border width
        )
    ),
    hoverinfo='none', text=list(G.nodes())
)

        axis = dict(
            showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')

        layout = go.Layout(width=800,  height=800,
                    scene=dict(xaxis=dict(axis),
                                yaxis=dict(axis),
                                zaxis=dict(axis)),
                                margin=dict(l=0, r=0, t=0, b=0)
                          )
        data = [ trace_nodes]
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(layout)
        fig.update_layout(layout)

        # Display the Plotly figure
        display(fig)
        display(k_slider)
        clear_output(wait=True)
    else:
        print('Lista de enlaces vacía')

k_slider = widgets.IntSlider(value=0, min=0, max=0, step=1, description='k:')

def slider_event_handler(change):
    update_graph(change.new)

k_slider.observe(slider_event_handler, names='value')

IntSlider(value=0, description='k:', max=3)

In [80]:
import json
with open('data/data.json', 'r') as json_file:
    puntos = json.load(json_file)

In [102]:
l=[k for k in puntos]
def best_match(n):
    min_l= min([abs(n-int(k)) for k in l])
    return [k for k in l if min_l==abs(n-int(k))][0]
best_match(90)    

'81'

In [99]:
l

['31',
 '151',
 '631',
 '2551',
 '101',
 '501',
 '2101',
 '8501',
 '801',
 '4001',
 '16801',
 '401',
 '2001',
 '8401',
 '81',
 '1681',
 '6801',
 '21',
 '421',
 '1701',
 '6821',
 '46',
 '226',
 '946',
 '3826',
 '1001',
 '5001',
 '21001']

In [17]:
enlaces=[(1,n) for n in range(800)]
G = nx.Graph()
G.add_edges_from(enlaces)
spring_3D = nx.spring_layout(G, dim=3, seed=1515, scale=10)

x_nodes = [spring_3D[i][0] for i in G.nodes()]
y_nodes = [spring_3D[i][1] for i in G.nodes()]
z_nodes = [spring_3D[i][2] for i in G.nodes()]

edge_list = G.edges()
x_edges, y_edges, z_edges = [], [], []

for edge in edge_list:
    x_coords = [spring_3D[edge[0]][0], spring_3D[edge[1]][0], None]
    x_edges += x_coords
    y_coords = [spring_3D[edge[0]][1], spring_3D[edge[1]][1], None]
    y_edges += y_coords
    z_coords = [spring_3D[edge[0]][2], spring_3D[edge[1]][2], None]
    z_edges += z_coords


trace_nodes = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers', marker=dict(
        symbol='circle',  # Use 'circle' symbol
        size=[5 for _ in list(G.nodes())],
        color=1,
        colorscale='Viridis',
        line=dict(
            color='black',  # Border color
            width=2          # Border width
        )
    ),
    hoverinfo='none', text=list(G.nodes())
)


axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')

layout = go.Layout(width=800,  height=800,
                    scene=dict(xaxis=dict(axis),
                                yaxis=dict(axis),
                                zaxis=dict(axis)),
                                margin=dict(l=0, r=0, t=0, b=0)
                          )
data = [ trace_nodes]
fig = go.Figure(data=data, layout=layout)
fig.update_layout(layout)

        # Display the Plotly figure
display(fig)

In [5]:
import plotly.graph_objects as go
import networkx as nx
import plotly.offline as pyo
import random
import copy
import igraph as ig
import ipywidgets as widgets
from IPython.display import display, clear_output
#inicializacion
n_usuarios, monto_inicial = 300, 15
n_iteraciones, n_referidos, monto_referir = 3, 4, 2.5
l_enlaces = []
graphs = []
#widgets
n_usuarios_input = widgets.IntSlider(value=n_usuarios, min=1, max=800, description='n_usuarios:')
monto_inicial_input = widgets.IntSlider(value=monto_inicial, min=1, max=100, description='monto_inicial:')
n_iteraciones_input = widgets.IntSlider(value=n_iteraciones, min=1, max=10, description='n_iteraciones:')
n_referidos_input = widgets.IntSlider(value=n_referidos, min=1, max=10, description='n_referidos:')
monto_referir_input = widgets.FloatSlider(value=monto_referir, min=0.1, max=10, step=0.1, description='monto_referir:')

run_button = widgets.Button(description='Run Simulation')
#output para ver los resultados
output = widgets.Output()

def refiere(de_nodo, n_monto, graph, enlaces):
    new_node_id = max(graph) + 1
    graph[new_node_id] = {'profundidad': graph[de_nodo]['profundidad'] + 1,'referidos':0, 'monto':monto_referir}
    graph[de_nodo]['referidos'] += 1
    enlaces.append((de_nodo,new_node_id))
    return graph, enlaces
def actualiza(graph, enlaces):
    c_graph=graph.copy()
    for k in c_graph:
        r=random.random()
        if graph[k]['profundidad']!=0 and graph[k]['referidos']<4 and r>0.5:
            n_referir=max(1,int((n_referidos +1)*random.random())-graph[k]['referidos'])
            for _ in range(n_referir):
                graph, enlaces=refiere(k, monto_referir, graph, enlaces)
    return graph, enlaces

def run_simulation(btn):
    global n_usuarios, monto_inicial, n_iteraciones, n_referidos, monto_referir
    global graphs, enlaces
    n_usuarios = n_usuarios_input.value
    monto_inicial = monto_inicial_input.value
    n_iteraciones = n_iteraciones_input.value
    n_referidos = n_referidos_input.value
    monto_referir = monto_referir_input.value
    with output:
        output.clear_output()
        graph = {1: {'profundidad': 0, 'referidos':0, 'monto':0}}
        enlaces=[]
        for _ in range(n_usuarios):
            n_id = max(graph) + 1
            graph[n_id] = {'profundidad': 1,'referidos':0, 'monto':monto_inicial}
            enlaces.append((1,n_id))
        c_graph = copy.deepcopy(graph)
        c_enlace = copy.deepcopy(enlaces)
        graphs.append(c_graph)
        l_enlaces.append(c_enlace)
        for step in range(n_iteraciones):
            graph, enlaces=actualiza(graph, enlaces)
            c_graph = copy.deepcopy(graph)
            c_enlace = copy.deepcopy(enlaces)
            graphs.append(c_graph)
            l_enlaces.append(c_enlace)
        k_slider.max = len(l_enlaces)-1
    update_graph(0)

run_button.on_click(run_simulation)

input_widgets = widgets.VBox([n_usuarios_input, monto_inicial_input, n_iteraciones_input,
                              n_referidos_input, monto_referir_input, run_button])
app_layout = widgets.HBox([input_widgets, output])
display(app_layout)

def calculate_aggregated_sum(graph):
    return sum(graph[k]['monto'] for k in graph)

def update_graph(k_value):
    global l_enlaces
    if 0 <= k_value < len(l_enlaces):
        G = nx.Graph()
        G.add_edges_from(l_enlaces[k_value])
        spring_3D = nx.spring_layout(G, dim=3, seed=1515, scale=10)

        x_nodes = [spring_3D[i][0] for i in G.nodes()]
        y_nodes = [spring_3D[i][1] for i in G.nodes()]
        z_nodes = [spring_3D[i][2] for i in G.nodes()]

        edge_list = G.edges()
        x_edges, y_edges, z_edges = [], [], []

        for edge in edge_list:
            x_coords = [spring_3D[edge[0]][0], spring_3D[edge[1]][0], None]
            x_edges += x_coords
            y_coords = [spring_3D[edge[0]][1], spring_3D[edge[1]][1], None]
            y_edges += y_coords
            z_coords = [spring_3D[edge[0]][2], spring_3D[edge[1]][2], None]
            z_edges += z_coords
        trace_edges = go.Scatter3d(
            x=x_edges, y=y_edges, z=z_edges,
                mode='lines',
                line=dict(
                color='rgb(125,125,125)',  # Color of the edges
                    width=1                    # Width of the edges
                        )
                    )
    

        trace_nodes = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers', marker=dict(
        symbol='circle',  # Use 'circle' symbol
        size=[5 for _ in list(G.nodes())],
        color=1,
        colorscale='Viridis',
        line=dict(
            color='black',  # Border color
            width=2          # Border width
        )
    ),
    hoverinfo='none', text=list(G.nodes())
)

        axis = dict(
            showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')

        layout = go.Layout(width=800,  height=800,
                    scene=dict(xaxis=dict(axis),
                                yaxis=dict(axis),
                                zaxis=dict(axis)),
                                margin=dict(l=0, r=0, t=0, b=0)
                          )
        data = [ trace_nodes, trace_edges]
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(layout)
        fig.update_layout(layout)

        # Display the Plotly figure
        display(fig)
        display(k_slider)
        clear_output(wait=True)
    else:
        print('Lista de enlaces vacía')

k_slider = widgets.IntSlider(value=0, min=0, max=0, step=1, description='k:')

def slider_event_handler(change):
    update_graph(change.new)

k_slider.observe(slider_event_handler, names='value')

IntSlider(value=0, description='k:', max=7)

In [5]:
#import pandas as pd
df=pd.read_csv('assets/BASE_DE_DATOS_CCMMNA_NIT-CIIU.csv')

In [9]:
df['ciiu'] = df['CIIU-1'].apply(lambda x: x.split('**')[0].strip())
entidades=df['RAZON SOCIAL'].unique().tolist()